# Optimize at fixed tuning parameter value(s)

`lsq_constrsparsereg.jl` fits constrained lasso

```math
\begin{split}
& \text{minimize} \hspace{1em} \frac 12||\boldsymbol{y}-\boldsymbol{X\beta}||^2_2 + \rho||\beta||_1 \\
& \text{ subject to} \hspace{0.5em} \boldsymbol{A\beta}=\boldsymbol{b} \text{ and } \boldsymbol{C\beta} \leq \boldsymbol{d}
\end{split}
```

at a fixed tuning parameter value ``\rho`` or several tuning parameter values provided by user.

### Single tuning parameter value

We demonstrate using a sum-to-zero constraint example

```math
\begin{split}
& \text{minimize} \hspace{1em} \frac 12||\boldsymbol{y}-\boldsymbol{X\beta}||^2_2 + \rho||\beta||_1 \\
& \text{ subject to} \hspace{0.5em} \sum_j \beta_j = 0
\end{split}
```

First, let's define a true parameter `β` such that `sum(β) = 0`.

In [37]:
using ConstrainedLasso, Base.Test

n, p = 50, 100  
β = zeros(p)
β[1:round(Int, p / 4)] = 0
β[(round(Int, p / 4) + 1):round(Int, p / 2)] = 1
β[(round(Int, p / 2) + 1):round(Int, 3p / 4)] = 0
β[(round(Int, 3p / 4) + 1):p] = -1
β

100-element Array{Float64,1}:
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  ⋮  
 -1.0
 -1.0
 -1.0
 -1.0
 -1.0
 -1.0
 -1.0
 -1.0
 -1.0
 -1.0
 -1.0
 -1.0

Next we generate data based on the true parameter `β`.

In [38]:
srand(41)
X = randn(n, p)

50×100 Array{Float64,2}:
  1.21212    -0.153889    0.141533  …  -0.458125    0.0951976  -2.14019   
  0.345895    1.30676     1.60944      -0.409901    0.323719    0.989333  
 -1.27859    -1.18894     0.512064      1.80509     1.62606    -1.44251   
  0.230616    2.54741    -0.523533      2.73358     1.07999     0.432834  
 -1.17103    -0.39082     0.441921     -0.179239   -0.158189   -0.640611  
  1.67135     0.0829011   0.964089  …  -0.720038    1.99359    -0.671572  
 -0.614717    2.16204    -0.0602       -0.324456   -0.616887    1.11243   
 -0.810535    0.974719   -0.045405      0.881578    1.29611     0.696869  
 -1.10879    -1.32489    -1.18272       0.579381   -0.971269   -0.687591  
 -0.219752   -0.447897   -0.974186     -0.880804   -0.480702   -1.36887   
  0.0952544  -0.126203   -0.273737  …  -0.264421    0.565684   -0.798719  
  1.4126      0.295896   -0.213161     -1.46343    -1.27144    -0.0589753 
 -0.418407   -0.479389    0.324243      1.96976     0.867659   -1.2999    


In [39]:
y = X * β + randn(n)

50-element Array{Float64,1}:
  -9.90585 
  -5.40562 
   5.24289 
  -6.29951 
  -4.9586  
  -6.1342  
  -7.90981 
   2.51009 
  -5.79548 
   1.61355 
  -0.722766
  10.4522  
   4.03935 
   ⋮       
   0.397781
  -2.6661  
   5.36896 
  -3.56537 
  -2.402   
   0.11478 
  -5.39248 
   4.38391 
   0.706801
 -10.1066  
  -1.12558 
  14.2473  

Since the equality constraint can be written as 

```math
\begin{split}
\begin{pmatrix} 1 & 1 & \cdots & 1 \end{pmatrix} \beta = 0,
\end{split}
```

we define the constraint as below.

In [40]:
beq   = [0.0]
Aeq   = ones(1, p)

1×100 Array{Float64,2}:
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  …  1.0  1.0  1.0  1.0  1.0  1.0  1.0

Now we are ready to fit the constrained lasso problem, say at `ρ=10`.

In [ ]:
ρ = 10.0
β̂, = lsq_constrsparsereg(X, y, ρ; Aeq = Aeq, beq = beq);

In [42]:
β̂

100×1 Array{Float64,2}:
  0.0334671 
 -1.1128e-5 
  5.40916e-6
 -0.394908  
  5.69496e-6
  1.17002e-5
 -3.90243e-6
 -1.91365e-5
  0.288553  
  2.05743e-6
  1.12622e-5
  1.6712e-5 
 -2.17275e-6
  ⋮         
 -0.188179  
 -3.52148e-6
  3.50838e-6
  9.58236e-6
 -0.844097  
 -0.645805  
 -0.257241  
 -4.90509e-6
 -1.13326   
 -7.28399e-6
  4.69294e-6
 -8.90143e-6

We see if the sum of estimated ``\beta`` coefficients equal to 0.

In [44]:
@test sum(β̂)≈0.0 atol=1e-4

Test Passed


### Multiple tuning parameter values

Define `ρlist` to be a sequence of values from 1 to 10.

In [26]:
ρlist = 152.0:-15.0:2.0

152.0:-15.0:2.0

Using the same equality constraints, we fit the constrained lasso.

In [ ]:
β̂, = lsq_constrsparsereg(X, y, ρlist; Aeq = Aeq, beq = beq);

In [28]:
β̂

100×11 Array{Float64,2}:
  1.01071e-5  -8.27382e-6  -9.98518e-7  …   2.24039e-6  -1.73712e-8
 -9.20705e-6  -1.73958e-5  -2.05881e-6     -2.60796e-5   0.178782  
 -5.63613e-6  -1.75901e-5  -1.8899e-6      -1.01686e-5   3.41563e-8
 -1.15136e-5  -1.29555e-5  -1.18262e-6     -0.331097    -0.413631  
 -1.60099e-5  -1.38074e-5  -1.11624e-6     -1.1681e-5    4.44159e-8
  1.6582e-5   -8.67409e-6  -7.82383e-7  …   2.73047e-5   1.19985e-7
 -1.06227e-5  -1.47899e-5  -7.36221e-7      4.90329e-6   5.63884e-8
 -5.7368e-6   -1.48341e-5  -1.65699e-6     -9.84382e-8   3.01081e-8
  1.42761e-5  -7.54447e-6  -9.46132e-7      0.205789     0.33502   
 -1.35551e-6  -1.40007e-5  -1.15001e-6      2.14801e-5  -0.158037  
 -8.39911e-6  -1.04558e-5  -1.59023e-6  …   4.57889e-6   1.88987e-8
  1.88431e-6  -1.34713e-5  -1.75855e-6     -2.03643e-6  -2.39007e-8
 -2.78397e-6  -9.21455e-6  -7.11068e-7     -1.14052e-6   3.91139e-8
  ⋮                                     ⋱                ⋮         
 -7.31054e-6  -1.5138e-

In [ ]:
using Plots; pyplot();
plot(ρlist, β̂', label="", xaxis = ("ρ", (minimum(ρlist),
      maximum(ρlist))), yaxis = ("β̂(ρ)"), width=0.5) 
title!("Solution Path at Fixed Parameter Values") 

In [36]:
savefig("misc/fixed.svg")